### Import Modules

In [1]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time
from importlib import reload

sys.path.append("../fitter/py_chiron")
import chiron

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [4]:
def FF(x):
    if isinstance(x, gv.GVar):
        f = chiron.FF(x.mean)
        stepSize = 1e-7
        dfdx = 0.5*(chiron.FF(x.mean+stepSize) - chiron.FF(x.mean-stepSize))/stepSize
        return gv.gvar_function(x, f, dfdx)

    else:
        return chiron.FF(x)

In [5]:
print(FF(0.2**2))
aG = gv.gvar(0.2, 0.1) ** 2
print(FF(aG))

0.5084416863714459
0.51(20)


In [3]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)

<module 'fitter.special_functions' from '../fitter/special_functions.py'>

### Specify fit parameters

In [168]:
p_dict = {
    'order' : {
        'fit' : 'nnlo', # 'nlo', 'nnlo', or 'nnnlo'
        'vol' : 10, # max 10
        'exclude' : [], # put LECs here
        'include_log' : True, # alphaS
        'include_log2' : True
    }, 
    'bs_N' : 1,  # if 0, use full list
    'fit_type' : 'ma', #'ma', 'ma-ratio, 'xpt', or 'xpt-ratio'; also 'simultaneous', 'ma-old'
    'F2' : 'FKFK', #'FKFpi', 'FpiFpi', 'FKFK', 'F0F0' (HPQCD choice)
    'chained' : False,
    'bias_correct' : True,
    'include_su2_isospin_corrrection' : False,
    
    'use_prior' : True,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'], # u'a15m130'
    
    'make_plots' : False,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,
    
    'save_results' : False,
}

In [169]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(fit_type=p_dict['fit_type'], F2=p_dict['F2'], include_FV=(p_dict['order']['vol']>0), 
                                 include_alphaS=p_dict['order']['include_log'], include_logSq=p_dict['order']['include_log2'])

# Make bootstrapper
bootstrapper = bs.bootstrapper(
    fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'], chain_fits=p_dict['chained'], 
    include_su2_isospin_corrrection=p_dict['include_su2_isospin_corrrection'],
    fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'], bias_correct=p_dict['bias_correct']
)

if p_dict['make_plots']:
    data_loader.save_plots(
        bootstrapper.make_plots(
            show_error_ellipses=p_dict['show_error_ellipses'], 
            show_bootstrap_histograms=p_dict['show_bs_histograms']),
        output_filename='fits/'+bootstrapper.fit_type+'_'+bootstrapper.order['fit']
    )
else:
    print(bootstrapper)
    
if p_dict['save_results']:
    data_loader.save_fit_info(bootstrapper.get_fit_info())
    
    
t1 = time.time()

print("\nTotal time (s): ", t1 - t0, "\n")

/mnt/Google/Documents/School Work/Research/fk-fpi/priors/ma.csv
Making fits...
Done!
100% complete
Time (s):  4.285393238067627
Compiling results...

Model: ma_FKFK_nnlo_FV_alphaS_logSq

Fitting to nnlo 
 with volume corrections O(10) 
 chained: False 
Fitted/[FLAG] values at physical point (including SU(2) isospin corrections: False):

F_K / F_pi = 1.1806(16) [1.1955(19)]   (delta_su2 = -0.00382(68))

  Parameter    Result[0] / Prior[1]    Parameter    Result[0] / Prior[1]
-----------------------------------  -----------------------------------
      A_a 0               0.28 (25)            1               0.00 (34)
          1               0.00 (34)        A_p 0               0.15 (17)
      A_k 0               0.08 (26)            1               0.00 (27)
          1               0.00 (27)        L_5 0          -0.000080 (59)
   A_loga 0               0.19 (30)            1             0.0000 (50)
---
boot0 fit results:
Least Square Fit:
  chi2/dof [dof] = 1.6 [16]    Q = 0.06   